In [9]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

import sys
sys.path.append("../scripts")
from data_cleaning import clean_data

In [11]:
#train
train_raw = pd.read_csv("../data/train.csv")
train_clean = clean_data(train_raw)
train_clean.to_csv("../data/train_clean.csv", index=False)

#test
test_raw = pd.read_csv("../data/test.csv")
test_clean = clean_data(test_raw)
test_clean.to_csv("../data/test_clean.csv", index=False)

print("Saved train_clean.csv and test_clean.csv")

KeyError: "['split', 'created_date', 'publication_id', 'parent_id', 'article_id', 'male', 'female', 'transgender', 'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual', 'other_sexual_orientation', 'christian', 'jewish', 'muslim', 'hindu', 'buddhist', 'atheist', 'other_religion', 'black', 'white', 'asian', 'latino', 'other_race_or_ethnicity', 'physical_disability', 'intellectual_or_learning_disability', 'psychiatric_or_mental_illness', 'other_disability', 'identity_annotator_count'] not found in axis"

In [ ]:
df = pd.read_csv('../data/train_clean.csv')
# df = pd.read_csv('../data/train.csv') # when running just the clean script instead of ./clean_scv

In [ ]:
# EDA

print(df['toxicity'].head())

print("\n")

print(df.columns)

print("\n")

print(np.min(df['toxicity']), np.max(df['toxicity']))


for idx, text in df["comment_text"].head(5).items():
    print(idx, text)

0    0.000000
1    0.000000
2    0.714286
3    0.000000
4    0.000000
Name: toxicity, dtype: float64


Index(['comment_text', 'funny', 'wow', 'sad', 'likes', 'disagree', 'toxicity',
       'severe_toxicity', 'obscene', 'sexual_explicit', 'identity_attack',
       'insult', 'threat', 'toxicity_annotator_count', 'rating_rejected'],
      dtype='object')


0.0 1.0
0 That was the reason Walker fire everyone now what ?.
1 So my original statement still stands . A church needs to not get involved in supporting Candidate A as opposed to Candidate B, it must however must make a stand on the side of the Gospel and help its congregation be informed and allow it to vote on that basis...
2 All you ever do is come to these forums berating white people with your Critical Race Theory garbage.

Please study Islamic Law before stupidly comparing someone you speculate is fueled by a verboten emotion to ISIS. 

Good grief, study something more than CRT.
3 Tonight will be an excellent opportunity for Clay

## SVC

In [23]:
# making label of toxicity feature into binary from continuous (0.0 to 1.0)

df["comment_text"] = df["comment_text"].fillna("").astype(str)
df["label"] = (df["toxicity"] >= 0.5).astype(int)

In [24]:
# chosen columns
# cols = ['rating', 'funny', 'wow', 'sad', 'likes',
#        'disagree', 'sexual_explicit',
#        'male', 'female', 'transgender',
#        'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual',
#        'other_sexual_orientation', 'christian', 'jewish', 'muslim', 'hindu',
#        'buddhist', 'atheist', 'other_religion', 'black', 'white', 'asian',
#        'latino', 'other_race_or_ethnicity', 'physical_disability',
#        'intellectual_or_learning_disability', 'psychiatric_or_mental_illness',
#        'other_disability']

text_col = "comment_text"
y = df["label"]

cols = [
    c for c in df.columns
    if c not in [text_col, "label", "toxicity"]
]

# df[cols] = df[cols].apply(pd.to_numeric, errors="coerce")
# df[cols] = df[cols].fillna(0)

print("Columns in cleaned df:", df.columns.tolist())
print("Numeric feature columns (first few):", cols[:10])

Columns in cleaned df: ['comment_text', 'funny', 'wow', 'sad', 'likes', 'disagree', 'toxicity', 'severe_toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat', 'toxicity_annotator_count', 'rating_rejected', 'label']
Numeric feature columns (first few): ['funny', 'wow', 'sad', 'likes', 'disagree', 'severe_toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult']


In [25]:
train_idx, val_idx = train_test_split(
    df.index,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

In [26]:
y_train = df["label"].loc[train_idx]
y_val = df["label"].loc[val_idx]

### Just text

In [27]:
# train test split (already done in data_prep.py and data_cleaning.py in ./scripts)
X_train_text = df.loc[train_idx, "comment_text"]
X_val_text   = df.loc[val_idx, "comment_text"]

In [28]:
# just LinearSVC with text in X_train or X_test
clf_text = Pipeline([
    ("tfidf", TfidfVectorizer( # strings to numeric
        stop_words="english",
        ngram_range=(1, 2),
        min_df=5,
        max_df=0.9
    )),
    ("svm", LinearSVC(random_state=42)) # regular linearSVC algorithm
])

clf_text.fit(X_train_text, y_train)
y_pred_text = clf_text.predict(X_val_text)

print(accuracy_score(y_val, y_pred_text))
print(classification_report(y_val, y_pred_text))


0.9464170270456871
              precision    recall  f1-score   support

           0       0.96      0.99      0.97    293147
           1       0.75      0.49      0.59     25499

    accuracy                           0.95    318646
   macro avg       0.86      0.74      0.78    318646
weighted avg       0.94      0.95      0.94    318646



### Other columns and text

In [29]:
X_train_both = df.loc[train_idx, ["comment_text"] + cols]
X_val_both   = df.loc[val_idx, ["comment_text"] + cols]

In [30]:
# LinearSVC with text and other columns
preprocess = ColumnTransformer(
    transformers=[
        ("text", TfidfVectorizer(
            stop_words="english",
            ngram_range=(1, 2)
        ), "comment_text"),
        ("num", StandardScaler(), cols),
    ]
)

clf_both = Pipeline([
    ("preprocess", preprocess),
    ("svm", LinearSVC(random_state=42))
])

clf_both.fit(X_train_both, y_train)
y_pred_both = clf_both.predict(X_val_both)

print(accuracy_score(y_val, y_pred_both))
print(classification_report(y_val, y_pred_both))

/Users/kaylakim/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.9824256384828305
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    293147
           1       0.90      0.87      0.89     25499

    accuracy                           0.98    318646
   macro avg       0.95      0.93      0.94    318646
weighted avg       0.98      0.98      0.98    318646



### Just other columns

In [31]:
X_train_num = df.loc[train_idx, cols]
X_val_num   = df.loc[val_idx, cols]

In [32]:
# LinearSVC withOUT text and just other columns
clf_num = Pipeline([
    ("ss", StandardScaler()),
    ("svm", LinearSVC(random_state=42))
])

clf_num.fit(X_train_num, y_train)
y_pred_num = clf_num.predict(X_val_num)

print(accuracy_score(y_val, y_pred_num))
print(classification_report(y_val, y_pred_num))

0.98381275773115
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    293147
           1       0.90      0.90      0.90     25499

    accuracy                           0.98    318646
   macro avg       0.95      0.94      0.94    318646
weighted avg       0.98      0.98      0.98    318646



## Optimizing just Both Comment and Columns SVC:

In [33]:
# proprocess and pipeline build
preprocess = ColumnTransformer(
    transformers=[
        ("text", TfidfVectorizer(
            stop_words="english"
        ), "comment_text"),
        ("num", StandardScaler(), cols),
    ]
)

base_svm_both = Pipeline([
    ("preprocess", preprocess),
    ("svm", LinearSVC(random_state=42, max_iter=10000))
])

In [ ]:
# Hyperparameter grid for combined model
param_grid_both = {
    "preprocess__text__ngram_range": [(1, 1), (1, 2)],
    "preprocess__text__min_df": [2, 5, 10],
    "svm__C": [0.1, 1, 5]
}

# actual optimization through GridSearchCV
grid_both = GridSearchCV(
    estimator=base_svm_both,
    param_grid=param_grid_both,
    cv=3, 
    scoring="f1",
    n_jobs=-1,
    verbose=2
)

# X_train_both and y_train from  unified split
grid_both.fit(X_train_both, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


In [ ]:
# print the parameters and scores for best parameters
print("Best params (combined SVM):", grid_both.best_params_)
print("Best CV F1 (combined SVM):", grid_both.best_score_)

best_svm_both = grid_both.best_estimator_

# Evaluate on the validation set
y_val_pred_both = best_svm_both.predict(X_val_both)

print("Validation accuracy (combined, tuned):", accuracy_score(y_val, y_val_pred_both))
print(classification_report(y_val, y_val_pred_both))

Best params (combined SVM): {'preprocess__text__min_df': 10, 'preprocess__text__ngram_range': (1, 1), 'svm__C': 1}
Best CV F1 (combined SVM): 0.5827025890088314
Validation accuracy (combined, tuned): 0.9443350463474377
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     38311
           1       0.74      0.47      0.57      3331

    accuracy                           0.94     41642
   macro avg       0.85      0.73      0.77     41642
weighted avg       0.94      0.94      0.94     41642

